### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import geocoder
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Creating dataset of postal codes

In [2]:
# Using BeautifulSoup get the list of postal codes of Canada from Wikipedia:

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [3]:
#Create function to cleanse lxml format

def td_clean(input_str):
    start = '<td>'
    end = '</td>'
    return str(input_str)[len(start):-len(end)-1]
Postals = My_table.findAll('td')

for i in range(len(Postals)):
    Postals[i] = td_clean(Postals[i])

    n_rows = len(Postals)/3
n_col = 3

In [4]:
#Create list of postcodes from lxml

PostalCodes = []
n = 0
for i in range(int(n_rows)):
    PostalCodes.append(Postals[n:n+3])
    n = n+3

In [5]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(data=PostalCodes, columns=column_names)
neighborhoods = neighborhoods.drop(neighborhoods[neighborhoods.Borough == 'Not assigned'].index).reset_index(drop  = True)
neighborhoods['Neighborhood'] = np.where(neighborhoods['Neighborhood'] == 'Not assigned', neighborhoods['Borough'], neighborhoods['Neighborhood'])
neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
#The shape of dataframe
neighborhoods.shape

(103, 3)

In [7]:
#Download data of latitude and longtitude and join tables

df_codes = pd.read_csv('https://cocl.us/Geospatial_data')
df_codes.set_index('Postal Code', inplace = True)

neighborhoods = neighborhoods.join(df_codes, on=["PostalCode"])
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [8]:
Toronto_nb = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]
Toronto_nb = Toronto_nb.drop('PostalCode',1)
Toronto_nb.head()

,Borough,Neighborhood,Latitude,Longitude
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,Downtown Toronto,St. James Town,43.651494,-79.375418
19,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_nb['Latitude'], Toronto_nb['Longitude'], Toronto_nb['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=Toronto_nb['Neighborhood'],
                                   latitudes=Toronto_nb['Latitude'],
                                   longitudes=Toronto_nb['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airp

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(1598, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


## 3. Analyze Each Neighborhood

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.016667,0.000000,0.016667,0.000000,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.024390,0.012195,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012195,...,0.012195,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

## 4. Cluster Neighborhoods

In [19]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 0, 9, 7, 6, 0, 6, 6, 0, 0])

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_nb

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Pub,Café,Farmers Market,Brewery,Shoe Store
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,6,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,6,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Pizza Place,Movie Theater
15,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Farmers Market,Cosmetics Shop,Department Store,Lingerie Store,Italian Restaurant
19,East Toronto,The Beaches,43.676357,-79.293031,5,Health Food Store,Trail,Pub,Wine Bar,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
The largest clusters are 0 and 6. The diversity of venues in clusters is low, making them too difficult to separate

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,Grocery Store,Café,Park,Italian Restaurant,Nightclub,Baby Store,Restaurant,Candy Store,Athletics & Sports,Coffee Shop
31,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Park,Pool,Bank,Bar,Café,Music Venue,Middle Eastern Restaurant,Supermarket
41,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Juice Bar,Bakery,Spa,Pub
43,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Burrito Place,Restaurant,Stadium,Bar,Italian Restaurant,Climbing Gym
47,"India Bazaar, The Beaches West",Park,Fast Food Restaurant,Sushi Restaurant,Movie Theater,Italian Restaurant,Fish & Chips Shop,Food & Drink Shop,Restaurant,Brewery,Ice Cream Shop
67,Davisville North,Gym / Fitness Center,Hotel,Park,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,General Entertainment,Deli / Bodega
69,"High Park, The Junction South",Mexican Restaurant,Café,Thai Restaurant,Park,Furniture / Home Store,Bookstore,Fast Food Restaurant,Flea Market,Speakeasy,Cajun / Creole Restaurant
74,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pizza Place,BBQ Joint,Pharmacy,Park,Liquor Store,Donut Shop,Burger Joint
75,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Restaurant,Movie Theater,Italian Restaurant,Bookstore,Dog Run,Dessert Shop,Bar,Cuban Restaurant
79,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Gym,Toy / Game Store,Seafood Restaurant


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Pub,Café,Farmers Market,Brewery,Shoe Store
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Café,Pizza Place,Movie Theater
15,St. James Town,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Farmers Market,Cosmetics Shop,Department Store,Lingerie Store,Italian Restaurant
20,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Basketball Stadium,Beach
24,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Thai Restaurant,Ramen Restaurant,Portuguese Restaurant
30,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Bakery,Gym,Thai Restaurant,Pizza Place,Hotel
36,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Sporting Goods Shop,Brewery,Scenic Lookout,Restaurant,Italian Restaurant,Fried Chicken Joint
37,"Little Portugal, Trinity",Bar,Café,Asian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Vietnamese Restaurant,Men's Store,Coffee Shop,Malay Restaurant,Italian Restaurant
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Salad Place,Restaurant,Italian Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,Gastropub
